In [ ]:
###############################################################################################################################
# Inferring Temporal Logic Specifications for Robot-Assisted Feeding in Social Dining Settings
#
# Jan Ondras (janko@cs.cornell.edu, jo951030@gmail.com)
# Project for Program Synthesis (CS 6172)
# Cornell University, Fall 2021
###############################################################################################################################
# Generate traces of features extracted from all three participants and the associated labels (of the target participant)
# I.e. generate the 'trio' dataset
# The generated traces and labels are directly used by the LoTuS formulae learning system
###############################################################################################################################

In [1]:
import numpy as np
import glob
import time
from scipy.io import savemat
import pandas as pd
import pympi
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler 

In [2]:
FEATURE_SOURCE = 'trio'

SAMPLING_FREQUENCY = 30. # FPS
SAMPLING_PERIOD = 1. / SAMPLING_FREQUENCY

input_features_filenames = np.array(sorted(glob.glob('/home/janko/projects/social-dining/data/processed/vision_openface_features/*.csv')))
input_annotations_filename_prefix = f'/home/janko/projects/social-dining/data/original/annotations/'
output_traces_filename_prefix = f'/home/janko/projects/social-dining/data/processed/traces/'

# Positive label is 1 and negative label is 2, as per the Matlab script
POS_LABEL = 1.
NEG_LABEL = 2.

# Map ELAN annotation tier name to desired label
eaf_tier_name_to_label = {
    'food_to_mouth': POS_LABEL
}

def generate_traces(selected_features, window_size):
    
    def get_features(features_file, target_labels):   
        # Load features
        df = pd.read_csv(features_file)

        # Add differential features if requested
        diff_features = dict()
        for feature_name in [ f[2:] for f in selected_features['features'] if f.startswith('d_') ]:
            diff_features['d_' + feature_name] = np.diff(df[feature_name], prepend=df[feature_name][0])  
        df = df.assign(**diff_features)

        # Add the time since last bite feature if requested
        if 'time_slb' in selected_features['features']:
            times_since_last_bite = []
            t = 0.
            for i in range(len(df)):
                if i < len(target_labels) and target_labels[i] == POS_LABEL:
                    t = 0.
                else:
                    t += SAMPLING_PERIOD
                times_since_last_bite.append(t)
            df = df.assign(time_slb=times_since_last_bite)
     
        # Keep only the selected features
        return df[selected_features['features']].to_numpy()


    traces_type = f'{FEATURE_SOURCE}_{selected_features["name"]}f_{window_size}w'
    print(f'Traces type: {traces_type}\tSelected features: {selected_features}')

    traces = []
    labels = []
    start_time = time.time()
    # Iterate over sessions
    assert len(input_features_filenames) % 3 == 0, "Sessions do not contain all three feature files!"
    for i in range(len(input_features_filenames) // 3):

        features_files_123 = input_features_filenames[(3 * i):(3 * i + 3)]
        session_id = features_files_123[0].split('/')[-1].split('.')[0].split('_')[0]
        print(f'Session ID: {session_id}')
        assert session_id == features_files_123[1].split('/')[-1].split('.')[0].split('_')[0], "Session not complete!"
        assert session_id == features_files_123[2].split('/')[-1].split('.')[0].split('_')[0], "Session not complete!"
        
        # For each session, there are 3 configurations of (left source) - (traget) - (right source)
        for j in range(3):
            features_file_target = features_files_123[j]
            features_file_right_source = features_files_123[(j + 1) % 3]
            features_file_left_source = features_files_123[(j + 2) % 3]
            
            # Definition: participant_id == session_id + '_' + participant_position
            target_participant_id = features_file_target.split('/')[-1].split('.')[0]
            print(f'Target participant ID: {target_participant_id}')
            assert len(target_participant_id) == 4

            # Get ELAN annotations for the target participant
            df_labels = np.full(len(pd.read_csv(features_file_target)), NEG_LABEL)
            eaf_obj = pympi.Elan.Eaf(f'{input_annotations_filename_prefix}{target_participant_id}.eaf')
            for tier_name in eaf_tier_name_to_label.keys():
                if tier_name not in eaf_obj.get_tier_names():
                    print(f"WARNING: no '{tier_name}' annotations found for Participant ID: {target_participant_id}")
                    assert False
                else:
                    # Iterate over annotations (times are in milliseconds)
                    for annotation in eaf_obj.get_annotation_data_for_tier(tier_name):
                        idx = int(SAMPLING_FREQUENCY * annotation[0] / 1000.)
                        assert idx < len(df_labels)
                        idx2 = int(SAMPLING_FREQUENCY * annotation[1] / 1000.)
                        df_labels[idx:idx2 + 1] = eaf_tier_name_to_label[tier_name]
                        last_positive_label_idx = idx2
                        
            # Get features for right and left source participants
            target_features = get_features(features_file_target, df_labels)
            right_source_features = get_features(features_file_right_source, df_labels)
            left_source_features = get_features(features_file_left_source, df_labels)
            features_len = min(len(target_features), len(right_source_features), len(left_source_features))

            # Convention: target features come first, then right source, then left source
            features = np.hstack((target_features[:features_len], right_source_features[:features_len], left_source_features[:features_len]))

            # Create labeled traces by sliding the window of length window_size over the features and labels
            for k in range(len(features) - window_size):
                # Skip data after the last bite (participant may have finished all their food => confusing data)
                if k + window_size > last_positive_label_idx:
                    break
                traces.append( features[k:k + window_size].flatten() )
                # Associate label of the next frame after the window (that is the reason why this loop iterates len(df) - window_size times)
                labels.append( df_labels[k + window_size] )

        print(f'\t => so far {len(traces)} traces ({Counter(labels)})')

    # Randomly undersample majority class
    # (this is the reason why we flatten each trace above and then reshape below)
    print("Label counts before random undersampling: ", Counter(labels))
    rus = RandomUnderSampler(random_state=42)
    traces, labels = rus.fit_resample(traces, labels)
    print("Label counts after random undersampling: ", Counter(labels))

    num_traces = len(traces)
    num_features = 3 * len(selected_features['features'])

    traces = np.swapaxes(np.reshape(traces, (num_traces, window_size, num_features)), 1, 2)
    timestamps = np.linspace(0., (window_size - 1) / SAMPLING_FREQUENCY, num=window_size).reshape((1, window_size))
    labels = np.reshape(labels, (num_traces, 1))

    assert traces.shape == (num_traces, num_features, window_size)
    assert timestamps.shape == (1, window_size)
    assert labels.shape == (num_traces, 1)

    # Save the traces as a Matlab .mat file
    struct = {
        'traces': traces,
        't':      timestamps,
        'labels': labels
    }
    savemat(f'{output_traces_filename_prefix}{traces_type}.mat', {'data': struct})

    print(f'Total number of traces: {len(traces)}')
    print(f'Total time taken: {time.time() - start_time} s')
    print('====================================================================================================')


In [3]:

selected_features = [
    {'name': 'tR2', 'features': [
        'time_slb',
        'pose_Rx', 
        'pose_Ry', 
    ]},

    {'name': 'R2', 'features': [
        'pose_Rx', 
        'pose_Ry', 
    ]},
    {'name': 'dR2', 'features': [
        'd_pose_Rx', 
        'd_pose_Ry', 
    ]},
    {'name': 'R2dR2', 'features': [
        'pose_Rx', 
        'pose_Ry', 
        'd_pose_Rx', 
        'd_pose_Ry', 
    ]},

    {'name': 'tR2dR2', 'features': [
        'time_slb',
        'pose_Rx', 
        'pose_Ry', 
        'd_pose_Rx', 
        'd_pose_Ry', 
    ]},




    # {'name': 'T3', 'features': [
    #     'pose_Tx', 
    #     'pose_Ty', 
    #     'pose_Tz',
    # ]},
    
    # {'name': 'R3', 'features': [
    #     'pose_Rx', # pitch
    #     'pose_Ry', # yaw
    #     'pose_Rz', # roll
    # ]},
    # {'name': 'dR3', 'features': [
    #     'd_pose_Rx', 
    #     'd_pose_Ry', 
    #     'd_pose_Rz',
    # ]},
    # {'name': 'R3dR3', 'features': [
    #     'pose_Rx', 
    #     'pose_Ry', 
    #     'pose_Rz',
    #     'd_pose_Rx', 
    #     'd_pose_Ry', 
    #     'd_pose_Rz',
    # ]},

    # {'name': 'T3R3', 'features': [
    #     'pose_Tx', 
    #     'pose_Ty', 
    #     'pose_Tz',
    #     'pose_Rx', 
    #     'pose_Ry', 
    #     'pose_Rz',
    # ]}
]

# window_sizes = [90]
window_sizes = [10, 20, 30, 60, 90, 120, 150, 180, 210]
window_sizes = [15, 45, 75, 105, 135, 165, 195]
window_sizes = [15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 210]
window_sizes = [60, 75]

for sf in selected_features:
    for ws in window_sizes:
        generate_traces(selected_features=sf, window_size=ws)

Traces type: trio_tR2f_60w	Selected features: {'name': 'tR2', 'features': ['time_slb', 'pose_Rx', 'pose_Ry']}
Session ID: 00
Target participant ID: 00_1
Target participant ID: 00_2
Target participant ID: 00_3
	 => so far 94272 traces (Counter({2.0: 91177, 1.0: 3095}))
Label counts before random undersampling:  Counter({2.0: 91177, 1.0: 3095})
Label counts after random undersampling:  Counter({1.0: 3095, 2.0: 3095})
Total number of traces: 6190
Total time taken: 18.3513400554657 s
Traces type: trio_tR2f_75w	Selected features: {'name': 'tR2', 'features': ['time_slb', 'pose_Rx', 'pose_Ry']}
Session ID: 00
Target participant ID: 00_1
Target participant ID: 00_2
Target participant ID: 00_3
	 => so far 94227 traces (Counter({2.0: 91132, 1.0: 3095}))
Label counts before random undersampling:  Counter({2.0: 91132, 1.0: 3095})
Label counts after random undersampling:  Counter({1.0: 3095, 2.0: 3095})
Total number of traces: 6190
Total time taken: 17.87565851211548 s
Traces type: trio_R2f_60w	Sel